In [2]:
!conda install -c anaconda beautifulsoup4 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.5.15  |                0         133 KB  anaconda
    certifi-2018.8.24          |           py35_1         139 KB  anaconda
    beautifulsoup4-4.6.3       |           py35_0         140 KB  anaconda
    openssl-1.0.2s             |       h7b6447c_0         3.1 MB  anaconda
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0      anaconda
    ca-certificates: 2019.1.23-0          --> 2019.5.15-0       anaconda
    certifi:         2018.8.24-py35_1     --> 2018.8.24-py35_1  anaconda
    openssl:   

In [3]:
!conda install -c anaconda lxml --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lxml-4.2.5                 |   py35hefd8a0e_0         1.6 MB  anaconda
    libxslt-1.1.33             |       h7d1a2b0_0         577 KB  anaconda
    libxml2-2.9.9              |       he19cac6_0         2.0 MB  anaconda
    ------------------------------------------------------------
                                           Total:         4.2 MB

The following packages will be UPDATED:

    libxml2: 2.9.4-h6b072ca_5     --> 2.9.9-he19cac6_0     anaconda
    libxslt: 1.1.29-hcf9102b_5    --> 1.1.33-h7d1a2b0_0    anaconda
    lxml:    4.1.0-py35ha401a81_0 --> 4.2.5-py35hefd8a0e_0 anaconda


lxml-4.2.5           | 1.6 MB    | ##################################### | 100% 
libxslt-1.1.33       | 577 KB   

In [4]:
from bs4 import BeautifulSoup
import requests

In [5]:
!wget -q -O 'postal_codes.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [6]:
with open('postal_codes.html') as html_file:
    soup = BeautifulSoup(html_file,'lxml')

In [7]:
table = soup.find('table', class_='wikitable sortable')

In [8]:
# try to scrape the webpage
for entry in table.find_all('th'):
    print(entry.text)

Postcode
Borough
Neighbourhood



In [9]:
import csv

In [10]:
# write the table data to a csv file
with open('postalcodes.csv', mode='w') as csv_file:
    csv_writer = csv.writer(csv_file)

    for entry in table.tbody.find_all('tr'):
        entry1 = entry.text.split('\n')[1]
        entry2 = entry.text.split('\n')[2]
        entry3 = entry.text.split('\n')[3]
        print(entry1, entry2, entry3)
        csv_writer.writerow([entry1, entry2, entry3])

Postcode Borough Neighbourhood
M1A Not assigned Not assigned
M2A Not assigned Not assigned
M3A North York Parkwoods
M4A North York Victoria Village
M5A Downtown Toronto Harbourfront
M5A Downtown Toronto Regent Park
M6A North York Lawrence Heights
M6A North York Lawrence Manor
M7A Queen's Park Not assigned
M8A Not assigned Not assigned
M9A Etobicoke Islington Avenue
M1B Scarborough Rouge
M1B Scarborough Malvern
M2B Not assigned Not assigned
M3B North York Don Mills North
M4B East York Woodbine Gardens
M4B East York Parkview Hill
M5B Downtown Toronto Ryerson
M5B Downtown Toronto Garden District
M6B North York Glencairn
M7B Not assigned Not assigned
M8B Not assigned Not assigned
M9B Etobicoke Cloverdale
M9B Etobicoke Islington
M9B Etobicoke Martin Grove
M9B Etobicoke Princess Gardens
M9B Etobicoke West Deane Park
M1C Scarborough Highland Creek
M1C Scarborough Rouge Hill
M1C Scarborough Port Union
M2C Not assigned Not assigned
M3C North York Flemingdon Park
M3C North York Don Mills South
M

In [11]:
import pandas as pd

In [12]:
df = pd.read_csv('postalcodes.csv')

In [13]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [14]:
# drop rows with Borough entry being 'Not assigned'
df = df[df.Borough != 'Not assigned']
df.reset_index(drop = True)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [15]:
# combine rows with same 'Postcode', 'Borough' and merge the data in 'Neighbourhood'
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [16]:
# if a 'Neighbourhood' entry is 'Not assigned', it will be replaced by the value of the 'Borough' colomn of the same row.
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df.loc[df['Neighbourhood'] == 'Not assigned', 'Borough']

In [17]:
df.sample(frac=0.1, random_state=1)

,Postcode,Borough,Neighbourhood
62,M5M,North York,"Bedford Park, Lawrence Manor East"
40,M4J,East York,East Toronto
39,M4H,East York,Thorncliffe Park
69,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade
102,M9W,Etobicoke,Northwest
38,M4G,East York,Leaside
51,M4X,Downtown Toronto,"Cabbagetown, St. James Town"
85,M7A,Queen's Park,Queen's Park
65,M5R,Central Toronto,"The Annex, North Midtown, Yorkville"
54,M5B,Downtown Toronto,"Ryerson, Garden District"


In [18]:
df.shape

(103, 3)